In [58]:
import matplotlib.pyplot as plt
import numpy as np

from astropy.io import fits

from astropy.visualization import ZScaleInterval, ImageNormalize, LinearStretch
import glob, os

In [59]:
from bokeh.io import show, output_notebook
# set local host for bokeh (change the number to the one in the url at the top of the page)
localhost = 'localhost:8888'

In [60]:
output_notebook()

Loading BokehJS ...

In [66]:
from photometryExercise import standardPlot
from photutils.aperture import aperture_photometry, CircularAperture, CircularAnnulus

In [62]:
pip install astropy

Note: you may need to restart the kernel to use updated packages.


In [63]:
# path settings
BASE_DIR = "data"

# standard stars + target fits files
STANDARD_DIR = os.path.join(BASE_DIR, "standards")

print(STANDARD_DIR)

# retrieving the data from the folders
standard_files = glob.glob(os.path.join(STANDARD_DIR, "*.fits"))

print(standard_files)

data/standards
['data/standards/PIRATE_OSL_ROE_EXO1_star1_Filter_I_1.fits']


In [64]:
# stolen from sid 
def show_fits(data, title=""):
    """
    stolen from the obs astro code but like the backend code they wrote which we didn't see
    """
    plt.figure(figsize=(8, 8))
    
    # Use ZScale for astronomical images
    interval = ZScaleInterval()
    vmin, vmax = interval.get_limits(data)
    norm = ImageNormalize(vmin=vmin, vmax=vmax, stretch=LinearStretch())
    
    plt.imshow(data, origin='lower', cmap='gray', norm=norm)
    plt.colorbar(label='Counts')
    plt.title(title)
    print(f"{title}: min={np.min(data):.1f}, max={np.max(data):.1f}, median={np.median(data):.1f}")
    print(f"Display range (ZScale): {vmin:.1f} to {vmax:.1f}")
    plt.show()
    
trim = 50  # idk what this should be lol

In [93]:
# a function that takes the total flux as input and returns a magnitude value

def flux_to_mag(C):
    return - 2.5 * np.log10(C)

In [94]:
# a function as well that estimates the error on the magnitude measuments

# C is the counts from the object
# s is the counts from the sky

def error_in_mag(C, s):

    # calculate the total counts within the aperture t
    t = C + s

    # calculate the final area of the objects counts by summing in quadrature of the errors
    del_C = np.sqrt(t + s)

    # calculate and return the magnitude error
    return - 2.5 / np.log(10) * (del_C / C)

In [95]:
# create an empty list of for the instrumental magnitudes and its corresponding error
instrumental_magnitude = []
instrumental_magnitude_error = []

In [96]:
# write a loop to obtain all the instrumental magnitudes and the errors 

for f in standard_files:

    standard = fits.open(f)[0].data

    #standardPlot(standard, notebook_url=localhost)

    # create the ciruclar aperture and sky annulus
    # find a way to automate this?? or do this properly within this loop??
    xcor = 1042
    ycor = 975
    rAperture = 15
    rSkyInner = 1.5 * rAperture
    rSkyOuter = 2 * rAperture

    apertureStandard = CircularAperture((xcor, ycor), r=rAperture)
    annulusSky = CircularAnnulus((xcor, ycor), r_in=rSkyInner, r_out=rSkyOuter)

    # make an estimation of the sky values in the defined annulus
    # aperture_photometry returns an astropy.table with the center of the aperture and 
    # the total flux on the aperture ('aperture_sum')
    sky = aperture_photometry(standard, annulusSky)
    #print(sky)

    # to recover the mean sky valye, weight the total flux by the area of the annulus

    area = np.pi * (rSkyOuter ** 2 - rSkyInner **2)
    skyFlux = sky['aperture_sum'].value
    meanSkyValue = skyFlux / area
    #print(f'The mean sky value is {meanSkyValue}')

    # remove the sky value
    standard = aperture_photometry(standard - meanSkyValue, apertureStandard)

    # extract value from astropy.table
    standardFlux = standard['aperture_sum'].value
    #print(f'The total flux on aperture is {standardFlux}')

    # calculating instrumental magnitude and its errors and append to list
    instrumental_magnitude.append(flux_to_mag(standardFlux))
    instrumental_magnitude_error.append(error_in_mag(standardFlux, skyFlux))

In [102]:
# print file name and found instrumental magnitude and its error
for i in range(0, len(standard_files)):

    # just tidy up the file name
    prefix_to_remove = "data/standards/"
    tidied_standard = standard_files[i].replace(prefix_to_remove, "")
    
    print(f"{tidied_standard} instrumental magnitude: {instrumental_magnitude[i][0]} ± {instrumental_magnitude_error[i][0]}")

PIRATE_OSL_ROE_EXO1_star1_Filter_I_1.fits instrumental magnitude: -13.41512655507815 ± -0.008662815192066657


In [54]:
# standard catalogue magntiudes for the four standard stars in each filter

# for standard star 1
standard_1_U = 0 
standard_1_B = 0
standard_1_V = 0
standard_1_I = 0

# for standard star 2
standard_2_U = 0
standard_2_B = 0
standard_2_V = 0
standard_2_I = 0

# for standard star 1
standard_2_U = 0
standard_2_B = 0
standard_2_V = 0
standard_2_I = 0

# for standard star 1
standard_2_U = 0
standard_2_B = 0
standard_2_V = 0
standard_2_I = 0

In [ ]:
# airmasses for the four standard stars in each filter

In [ ]:
# calculate the extinction coefficients

#for f in in instrumental_magnitude:
    